In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

conn = sqlite3.connect('/Users/beans/Desktop/TeamHousing/data/processed/nyc_demolitions.db')
cursor = conn.cursor()

queryowner_demo = """
SELECT *
FROM fact_demolitions f   
"""

fact_owner_demo = pd.read_sql_query(queryowner_demo, conn)

queryshelter = """
SELECT *
FROM fact_shelters
"""
fact_shelter = pd.read_sql_query(queryshelter, conn)

display(fact_owner_demo.sort_values(by=['bin'],ascending=False).head())
display(fact_shelter.sort_values(by=['report_date'],ascending=False).head())

# setting job_typeid to strings to represent what the job done is
mapdemo = {1:'construction',2:'demolition'}
fact_owner_demo['job_typeid'] = fact_owner_demo['job_typeid'].map(mapdemo)

# setting ownership_id 1 to private and 2/3 to affordable (government & nonprofit)
mapowner = {1:'private',2:'affordable',3:'affordable'}
fact_owner_demo['ownership_id'] = fact_owner_demo['ownership_id'].map(mapowner)

fact_owner_demo.count()

,month_date,bin,job_typeid,ownership_id,borough,date_filed,date_completed,time_of_completion
13032,2023-03-01 00:00:00,5869568,2,1,Staten Island,2022-11-18 00:00:00,2023-03-13 00:00:00,115
12397,2018-08-01 00:00:00,5867189,2,1,Staten Island,2018-06-21 00:00:00,2018-08-03 00:00:00,43
12395,2018-09-01 00:00:00,5867187,2,1,Staten Island,2018-06-21 00:00:00,2018-09-24 00:00:00,95
12394,2018-11-01 00:00:00,5867186,2,1,Staten Island,2018-06-21 00:00:00,2018-11-26 00:00:00,158
12393,2018-11-01 00:00:00,5867185,2,1,Staten Island,2018-06-21 00:00:00,2018-11-26 00:00:00,158


,report_date,shelter_count
106,2024-12-01 00:00:00,67298
107,2024-11-01 00:00:00,66981
108,2024-10-01 00:00:00,67218
109,2024-09-01 00:00:00,68336
110,2024-08-01 00:00:00,68549


month_date            15364
bin                   15364
job_typeid            15364
ownership_id          15364
borough               15364
date_filed            15364
date_completed        15364
time_of_completion    15364
dtype: int64

In [3]:
# table plots all constructions and demolitions by borough
# aggregation counts totals for each job type (construction or demolition) & average time_of_completion per job type
# FINDINGS: takes a lot longer for construction than demolition between private and affordable
boroughagg = fact_owner_demo.groupby(['ownership_id','job_typeid']).agg({'month_date':'count','time_of_completion':'mean'}).reset_index()

boroughagg

,ownership_id,job_typeid,month_date,time_of_completion
0,affordable,construction,694,1554.889049
1,affordable,demolition,302,161.311258
2,private,construction,10468,1579.296905
3,private,demolition,3900,228.393846


In [4]:
# im curious to know if demolition/construction is mutually exclusive (gonna use the original csv for the unique identifier)
# duplicated BIN (should be unique building identifier)

housingcsv = pd.read_csv('/Users/beans/Desktop/TeamHousing/data/HousingDB_post2010.csv')

housingcsv = housingcsv[housingcsv['Ownership'].notna()]

housingcsv['ownership_clean'] = housingcsv['Ownership'].str.strip()

housingcsv['DateComplt'] = pd.to_datetime(housingcsv['DateComplt'])


housingcsv = housingcsv[
    (housingcsv['Job_Type'].isin(['New Building', 'Demolition'])) &
    (housingcsv['Job_Status'] == '5. Completed Construction') &
    (housingcsv['ResidFlag'] == 'Residential') &
    (housingcsv['NonresFlag'].isna()) &
    (housingcsv['DateFiled'].notna()) &
    (housingcsv['DateComplt'].notna()) &
    (housingcsv['DateComplt'].dt.year.between(2016, 2024))
]

# 3064 DUPLICATED BIN hmmm
housingcsv['BIN'].duplicated().sum()

# removing duplicates to see if changes spread of data
#housingcsv.drop_duplicates(subset=['BIN'],inplace=True)

# quick clean (from original schema file )
housingcsv['Ownership'] = housingcsv['ownership_clean'].apply(lambda x: x.split(':')[0])
housingcsv['Ownership'] = housingcsv['Ownership'].apply(lambda x: x.split(',')[0])

csvmap = {'Private For-Profit':'Not Affordable','Government':'Affordable','Private Non-Profit':'Affordable'}
housingcsv['Ownership'] = housingcsv['Ownership'].map(csvmap)
# doing same groupby as code block above, trying to visualize if spread has changed
# FINDINGS: PROPORTION HAS DEFINITELY CHANGED, AFFORDABLE PROPORTION OF DEMOLITION IS 
housingcsv.groupby(['Ownership','Job_Type']).agg({'Job_Number':'count'})


# JUSTIFYING IF WE SHOULD REMOVE DUPLICATED
housingcsv['trueflag'] = housingcsv['BIN'].duplicated()
demodup = housingcsv[(housingcsv['trueflag'] == 1)]

display(demodup['BIN'].head)


# three dups
duptestone = demodup[demodup['BIN'] == 2130391]
display(duptestone[['Boro','Ownership','Job_Type','AddressNum','AddressSt','DateFiled','DatePermit','DateComplt','BIN']])

# this one is weird
duptesttwo = demodup[demodup['BIN'] == 2094743]
display(duptesttwo[['Boro','Ownership','Job_Type','AddressNum','AddressSt','DateFiled','DatePermit','DateComplt','BIN']])

# test 3 has different dates in completed, strange
duptesthree = demodup[demodup['BIN'] == 1046469]
display(duptesthree[['Boro','Ownership','Job_Type','AddressNum','AddressSt','DateFiled','DatePermit','DateComplt','BIN']])

# ENDED UP JUSTIFYING THE REMOVAL OF DUPLICATES
# REASON : WE ARE DOING OUR ANALYSIS ON A BUILDING LEVEL/ NOT UNIT LEVEL. ONE DUPLEX/TRIPLEX COUNTS FOR ONE BUILDING
# (AVOIDS COUNTING ONE BUILDING MULTIPLE TIMES)

/var/folders/3y/ldxff8k17wjcyzkq19xw1srm0000gn/T/ipykernel_70949/1523067499.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  housingcsv['DateComplt'] = pd.to_datetime(housingcsv['DateComplt'])


<bound method NDFrame.head of 82       2042556
182      2130391
205      2130391
206      2130391
212      2129192
          ...   
79119    2130362
79120    2013665
79121    2129142
79122    2005406
79416    2094743
Name: BIN, Length: 3064, dtype: int64>

,Boro,Ownership,Job_Type,AddressNum,AddressSt,DateFiled,DatePermit,DateComplt,BIN
182,2,Not Affordable,Demolition,2092,VALENTINE AVENUE,12/17/15,1/5/16,2016-01-05,2130391
205,2,Not Affordable,Demolition,2088,VALENTINE AVENUE,12/17/15,1/5/16,2016-01-05,2130391
206,2,Not Affordable,Demolition,2086,VALENTINE AVENUE,12/18/15,1/5/16,2016-01-05,2130391
1212,2,Not Affordable,New Building,2086,VALENTINE AVENUE,4/18/19,6/11/20,2022-03-22,2130391


,Boro,Ownership,Job_Type,AddressNum,AddressSt,DateFiled,DatePermit,DateComplt,BIN
79416,2,Not Affordable,Demolition,3055,VALENTINE AVENUE,4/23/15,6/22/16,2016-06-22,2094743


,Boro,Ownership,Job_Type,AddressNum,AddressSt,DateFiled,DatePermit,DateComplt,BIN
76730,1,Not Affordable,Demolition,1045,MADISON AVENUE,9/11/18,2/13/19,2019-02-13,1046469
76731,1,Not Affordable,Demolition,1043,MADISON AVENUE,9/11/18,2/4/19,2019-02-04,1046469
76732,1,Not Affordable,Demolition,1041,MADISON AVENUE,9/11/18,2/7/19,2019-02-07,1046469


In [5]:
# PROPORTIONS AFTER REMOVING DUPLICATES
print('affordable proportion',302/(302+694))
print('not affordable proportion',3902/(10468+3902))

affordable proportion 0.3032128514056225
not affordable proportion 0.2715379262352122


# PLOTS FOR FURTHER EDA

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

scaleddata = fact_owner_demo[['month_date','job_typeid','ownership_id']]



#fact_owner_demo.groupby(['month_date','job_typeid','ownership_id'])['job_typeid'].agg('count')

SyntaxError: invalid syntax (993390429.py, line 3)

# ANOVA TEST TO SEE DIFFERENCE BETWEEN PROPORTIONS OF BOROUGHS

### might need to adjust what this ANOVA is checking...

In [39]:
# filter for all demolitions (looking to find any trends demolition specific)
demolitions = fact_owner_demo[fact_owner_demo['job_typeid'] == 'demolition']
# able to see time of completion is quickest in staten island
demolitions.groupby('borough').agg({'time_of_completion':'mean'})

demolitions['borough'].value_counts()

# grouping fact table by jop_typeid (construction/demolition), ownership_id (affordable/private), and borough
# counting bin as it'll represent each unique building
boroughwork = fact_owner_demo.groupby(['job_typeid','ownership_id','borough']).agg({'bin':'count'}).sort_values(by=['ownership_id','borough']).reset_index()

display(boroughwork)

# filtering to total demolitions
totaldemolitions = boroughwork[boroughwork['job_typeid'] == 'demolition'].reset_index()
# creating a list for all total demolitions
listtotaldem = totaldemolitions['bin']
# creating a total buildings dataframe to have just ownership_id and borough & sum of buildings
totalbuildings = boroughwork.groupby(['ownership_id','borough']).agg({'bin':'sum'}).reset_index()

totalbuildings['totaldem'] = listtotaldem

totalbuildings['proportion'] = totalbuildings['totaldem'] / totalbuildings['bin']

totalbuildings.rename(columns={'bin':'totalbuildings'},inplace=True)

display(totalbuildings)

,job_typeid,ownership_id,borough,bin
0,construction,affordable,Bronx,90
1,demolition,affordable,Bronx,40
2,construction,affordable,Brooklyn,451
3,demolition,affordable,Brooklyn,81
4,construction,affordable,Manhattan,40
5,demolition,affordable,Manhattan,15
6,construction,affordable,Queens,96
7,demolition,affordable,Queens,98
8,construction,affordable,Staten Island,17
9,demolition,affordable,Staten Island,68


,ownership_id,borough,totalbuildings,totaldem,proportion
0,affordable,Bronx,130,40,0.307692
1,affordable,Brooklyn,532,81,0.152256
2,affordable,Manhattan,55,15,0.272727
3,affordable,Queens,194,98,0.505155
4,affordable,Staten Island,85,68,0.800000
5,private,Bronx,1575,625,0.396825
6,private,Brooklyn,4348,1136,0.261270
7,private,Manhattan,675,176,0.260741
8,private,Queens,4152,1240,0.298651
9,private,Staten Island,3618,723,0.199834


In [ ]:
# takes each borough & groups by proportion
# creates a dictionary of lists (for ANOVA)
borough_lists = (
    totalbuildings
    .groupby('borough')['proportion']
    .apply(list)
    .to_dict()
)

borough_lists

{'Bronx': [0.3076923076923077, 0.3968253968253968],
 'Brooklyn': [0.15225563909774437, 0.2612695492180313],
 'Manhattan': [0.2727272727272727, 0.2607407407407407],
 'Queens': [0.5051546391752577, 0.29865125240847784],
 'Staten Island': [0.8, 0.1998341625207297]}

In [106]:
# Null Hypothesis: 
# The average proportions of demolition/total buildings for affordability and private are the same
# Alternate Hypothesis:
# At least one group's mean is different from the others.

from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd

manhattan = borough_lists['Manhattan']
bronx = borough_lists['Bronx']
brooklyn = borough_lists['Brooklyn']
queens = borough_lists['Queens']
stateni = borough_lists['Staten Island']

# perform one way ANOVA
f_statistic, p_value = f_oneway(manhattan, bronx, brooklyn, queens, stateni)

print('Our P-Value for our One-Way ANOVA is :',p_value)
print('This means we must reject our Alternate Hypothesis, and our Null Hypothesis is true.\n')

print("To make sure, I did a Tukey's Test:")
# create a dataframe with all values for tukey's test
data = manhattan + bronx + brooklyn + queens + stateni
labels = ['manhattan']*len(manhattan) + ['bronx']*len(bronx) + ['brooklyn']*len(brooklyn) + ['queens']*len(queens) + ['stateni']*len(stateni)

# Perform Tukey's Test
tukey_results = pairwise_tukeyhsd(endog=data, groups=labels, alpha=0.05)
print(tukey_results,"\n Our Tukey's Test confirms there are no statistically significant differences between borough proportions.")


Our P-Value for our One-Way ANOVA is : 0.6674873008601531
This means we must reject our Alternate Hypothesis, and our Null Hypothesis is true.

To make sure, I did a Tukey's Test:
   Multiple Comparison of Means - Tukey HSD, FWER=0.05   
  group1    group2  meandiff p-adj   lower  upper  reject
---------------------------------------------------------
    bronx  brooklyn  -0.1455  0.946 -0.9704 0.6794  False
    bronx manhattan  -0.0855 0.9918 -0.9104 0.7393  False
    bronx    queens   0.0496  0.999 -0.7752 0.8745  False
    bronx   stateni   0.1477 0.9434 -0.6772 0.9725  False
 brooklyn manhattan     0.06 0.9979 -0.7649 0.8848  False
 brooklyn    queens   0.1951 0.8668 -0.6297   1.02  False
 brooklyn   stateni   0.2932 0.6399 -0.5317  1.118  False
manhattan    queens   0.1352 0.9578 -0.6897   0.96  False
manhattan   stateni   0.2332 0.7853 -0.5917  1.058  False
   queens   stateni    0.098 0.9864 -0.7269 0.9229  False
--------------------------------------------------------- 
 Our Tu

# TWO SAMPLE T-TEST OF PROPORTIONS VS PROPORTION (borough level)

In [117]:
# Import the necessary library
from scipy import stats

affordable = totalbuildings[totalbuildings['ownership_id'] == 'affordable']
private = totalbuildings[totalbuildings['ownership_id'] == 'private']

# Perform the independent t-test (Welch's t-test)
# We use equal_var=False because we assume the group variances might not be equal
result = stats.ttest_ind(private['proportion'], affordable['proportion'], equal_var=False, alternative='greater')

# Print the results
print(f"t-statistic: {result.statistic:.2f}")
print(f"p-value: {result.pvalue:.4f}")

# Make a decision based on a common significance level (e.g., alpha = 0.05)
alpha = 0.05
if result.pvalue < alpha:
   print("\nConclusion: The proportion of borough level demolitions of private housing to total private housing is greater than borough level proportions\n of demolished affordable housing to total affordable housing")
else:
   print("\nConclusion: The proportion of borough level demolitions of private housing to total private housing is not greater than borough level proportions\n of demolished affordable housing to total affordable housing")

t-statistic: -1.05
p-value: 0.8280

Conclusion: The proportion of borough level demolitions of private housing to total private housing is not greater than borough level proportions
 of demolished affordable housing to total affordable housing
